# Something unexpected behavior on 2024-11-26

The expected behavior: 
the SV field `Rubin_SV_38_7` should be observed in a 2x2 grid dither pattern, with equal coverage on each corner.

The observed behavior: 
the field received about the expected number of visits in each band (according to the next-day version of the configuration), but in the wrong places in the dither pattern.

In [1]:
import os
import numpy as np
import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.summit.utils import ConsDbClient

import sys
sys.path.insert(0, '/home/l/lynnej/u/repos/rubin_scheduler')

import pickle
from urllib.parse import urlparse
from lsst.resources import ResourcePath
from lsst_efd_client import EfdClient
from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site
from rubin_scheduler.scheduler.example import get_ideal_model_observatory

from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler.features import Conditions

os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/shared/rubin_sim_data'

days_to_seconds = 24*60*60

In [5]:
# Verify visits acquired. 

os.environ["LSST_CONSDB_PQ_URL"] = "http://consdb-pq.consdb:8080/consdb"
os.environ["no_proxy"] += ",.consdb"

day_obs = "2024-11-26"

consdb = ConsDbClient()
instrument = 'lsstcomcam'
day_obs_int = day_obs.replace('-', '')
visit_query = f'''
    SELECT * FROM cdb_{instrument}.visit1
     where day_obs = {day_obs_int}
'''
visits = consdb.query(visit_query).to_pandas()

target = "Rubin_SV_38_7"
block = "BLOCK-320"

vv = pd.DataFrame(visits.query('science_program == "BLOCK-320" and target_name == @target'))
delta_visit = vv.exp_midpt_mjd.values[1:] - vv.exp_midpt_mjd.values[:-1]
delta_visit = np.concatenate([np.array([0]), delta_visit])
vv['delta_visit'] = delta_visit * 24 * 60 * 60
print(vv.exp_midpt_mjd.min(), vv.exp_midpt_mjd.max(), (vv.exp_midpt_mjd.max() - vv.exp_midpt_mjd.min()) * 24 * 60)
vcols = ['exposure_name', 'group_id', 'delta_visit', 'exp_midpt', 'exp_midpt_mjd', 'band', 's_ra', 's_dec', 'sky_rotation', 'altitude', 'azimuth', 'target_name',]
display(HTML(vv[vcols].to_html()))
display(vv[['exp_midpt', 'band', 'target_name']].groupby(['target_name', 'band']).count())

60641.04732438143 60641.08242280715 50.541733042337


,exposure_name,group_id,delta_visit,exp_midpt,exp_midpt_mjd,band,s_ra,s_dec,sky_rotation,altitude,azimuth,target_name
102,CC_O_20241126_000103,2024-11-27T01:07:33.217,0.000000,2024-11-27T01:08:08.826500,60641.047324,r,38.352611,7.094078,211.836837,45.052705,38.081810,Rubin_SV_38_7
103,CC_O_20241126_000104,2024-11-27T01:07:34.366,50.682845,2024-11-27T01:08:59.509000,60641.047911,r,37.975071,6.491146,211.836203,44.230145,37.835794,Rubin_SV_38_7
104,CC_O_20241126_000105,2024-11-27T01:08:35.963,51.465038,2024-11-27T01:09:50.974500,60641.048507,r,37.367630,6.865412,211.836191,44.120022,36.556651,Rubin_SV_38_7
105,CC_O_20241126_000106,2024-11-27T01:09:25.966,52.330546,2024-11-27T01:10:43.305000,60641.049112,r,37.744689,7.468826,211.835878,44.717645,36.304401,Rubin_SV_38_7
106,CC_O_20241126_000107,2024-11-27T01:10:18.374,55.001867,2024-11-27T01:11:38.307000,60641.049749,r,38.393965,7.112703,211.836182,44.629106,37.073689,Rubin_SV_38_7
107,CC_O_20241126_000108,2024-11-27T01:11:10.521,53.415802,2024-11-27T01:12:31.722500,60641.050367,r,38.016360,6.509805,211.836230,43.803291,36.794792,Rubin_SV_38_7
108,CC_O_20241126_000109,2024-11-27T01:12:04.768,52.499978,2024-11-27T01:13:24.222500,60641.050975,r,37.408925,6.884123,211.836076,43.704272,35.498094,Rubin_SV_38_7
109,CC_O_20241126_000110,2024-11-27T01:12:59.032,54.890142,2024-11-27T01:14:19.112500,60641.051610,r,37.786055,7.487505,211.836045,44.299227,35.244236,Rubin_SV_38_7
110,CC_O_20241126_000111,2024-11-27T01:13:50.985,97.774621,2024-11-27T01:15:56.887000,60641.052742,g,38.352611,7.094078,211.836132,44.038171,35.723927,Rubin_SV_38_7
111,CC_O_20241126_000112,2024-11-27T01:14:46.340,51.466159,2024-11-27T01:16:48.354000,60641.053337,g,37.975071,6.491146,211.836065,43.220413,35.429304,Rubin_SV_38_7


exp_midpt
target_name   band           
Rubin_SV_38_7 g            12
              i            14
              r             8

In [7]:
# Set a range of times based on these visits values to query efd and obsenv .. 

t_start = Time("2024-11-27T01:07:33.217	", format='isot', scale='utc') - TimeDelta(15 * 60 , format='sec')
t_end = Time("2024-11-27T01:57:25.311", format='isot', scale='utc') + TimeDelta(10 * 60, format='sec')

print(f"Will query from {t_start.iso} to {t_end.iso}")

Will query from 2024-11-27 00:52:33.217 to 2024-11-27 02:07:25.311


In [9]:
efd = 'usdf_efd'

efd_client = EfdClient(efd)
obsenv_client = EfdClient(efd, db_name='lsst.obsenv')

In [36]:
# find the obsenv in place at that time, if we want to retrieve configurations

# Scheduler dependency information -- needs to go into package 
async def get_scheduler_configs(t_start: Time, t_end: Time, efd_client: EfdClient, obsenv_client: EfdClient | None) -> pd.DataFrame:
    # Scheduler dependency information
    t_start_local = t_start
    topic = 'lsst.sal.Scheduler.logevent_dependenciesVersions'
    fields = await efd_client.get_fields(topic)
    fields = [f for f in fields if "private" not in f]
    deps = await efd_client.select_time_series(topic, fields, t_start_local, t_end)
    # Sometimes the scheduler hasn't been set up, if it's a limited timespan.
    if len(deps) == 0:
        t_start_local = t_start - TimeDelta(1, format='jd')
        deps = await efd_client.select_time_series(topic, fields, t_start_local, t_end)
        deps = deps.iloc[:1]
    # Reconfigure output to fit into script_status fields 
    deps['classname'] = "Scheduler dependencies"
    deps['description'] = deps['scheduler'] + ' ' + deps['seeingModel']
    models = [c for c in deps.columns if 'observatory' in c or 'Model' in c]
    def build_dep_string(x, models): 
        dep_string = ''
        for m in models:
            dep_string += f"{m}: {x[m]}, "
        dep_string = dep_string[:-2]
        return dep_string
    deps['config'] = deps.apply(build_dep_string, args=[models], axis=1)
    deps['script_salIndex'] = -1
    
    # And within Scheduler, what is ts_config_ocs and scripts versions
    # Need find the previous version of tc_config_ocs 
    topic = 'lsst.obsenv.summary'
    fields = ['summit_extras', 'ts_standardscripts', 'ts_externalscripts', 'ts_config_ocs']
    # Query longer time period for obsenv, so we can be sure to know how scheduler enables
    obsenv = await obsenv_client.select_time_series(topic, fields, t_start_local - TimeDelta(1, format='jd'), t_end)
    fields = ['summit_extras', 'ts_standardscripts', 'ts_externalscripts', 'ts_config_ocs']
    check = np.all((obsenv[fields][1:].values == obsenv[fields][:-1].values), axis=1)
    classname = np.where(check, "Obsenv Check", "Obsenv Update")
    obsenv['classname'] = np.concatenate([np.array(['Obsenv']), classname])
    obsenv['description'] = ("ts_config_ocs: " + obsenv['ts_config_ocs'] + 
                            " summit_extras: " + obsenv['summit_extras'])
    obsenv['config'] = ("ts_standardscripts: " + obsenv['ts_standardscripts'] + 
                        " ts_externalscripts: " + obsenv['ts_externalscripts'])
    obsenv['salIndex'] = 1
    obsenv['script_salIndex'] = -1
    
    # I think these should be every time scheduler is "ENABLED"
    topic = 'lsst.sal.Scheduler.logevent_configurationApplied'
    fields = await efd_client.get_fields(topic)
    fields = [f for f in fields if "private" not in f]
    con = await efd_client.select_time_series(topic, fields, t_start_local, t_end)
    con['classname'] = "Scheduler configuration"
    # Build description from schemaVersion (just in case) and ts_config_ocs 
    ts_config_ocs_in_place = []
    for time in con.index:
        prev_obsenv = obsenv.query('index < @time')
        if len(prev_obsenv) == 0:
            ts_config_ocs_in_place.append('Unknown')
        else:
            ts_config_ocs_in_place.append(prev_obsenv.iloc[-1]['ts_config_ocs'])
    con['ts_config_ocs'] = ts_config_ocs_in_place
    con['description'] = 'ts_config_ocs ' + con['ts_config_ocs'] + ' ' + con['schemaVersion']
    con.rename({'configurations': 'config'}, axis=1, inplace=True)
    con['script_salIndex'] = -1

    # Combine results
    dd =  pd.concat([deps, con, obsenv])
    # Trim back results to t_start, keeping last previous update information
    # Trim obsenv back to range for other values
    # But keep last entry so we have easy record 
    tt = pd.to_datetime(t_start.utc.datetime).tz_localize("UTC")
    # Keep last scheduler configuration update
    old_dd_sched = dd.query('index < @tt and classname == "Scheduler configuration"')[-1:]
    old_dd_deps = dd.query('index < @tt and classname == "Scheduler dependencies"')[-1:]
    old_dd_obsenv = dd.query('index < @tt and classname.str.contains("Obsenv")')[-1:]
    dd = dd.query('index >= @tt')
    sched_config = pd.concat([old_dd_sched, old_dd_obsenv, old_dd_deps, dd])

    # Reformat
    cols = ['classname', 'description', 'config', 'salIndex', 'script_salIndex']
    drop_cols = [c for c in sched_config.columns if c not in cols]
    sched_config.drop(drop_cols, axis=1, inplace=True)
    sched_config.sort_index(inplace=True)
    sched_config['timestampProcessStart'] = sched_config.index.copy().tz_localize(None).astype('datetime64[ns]')
    sched_config['finalScriptState'] = "Configuration"
    print(f"Found {len(sched_config)} scheduler configuration records")
    return sched_config

dd = await get_scheduler_configs(t_start, t_end, efd_client, obsenv_client)
display(dd)
# ts_config_ocs link:
ts_config_ocs_githash = dd.query('classname=="Scheduler configuration"')['description'].values[0].split(' ')[1].lstrip('v')
fbs_config = dd.query('classname == "Scheduler configuration"')['config'].values[0].split(',')[-1]
link = f"https:/github.com/lsst-ts/ts_config_ocs/tree/{ts_config_ocs_githash}"
display(Markdown(f"fbs configuration yaml @ {fbs_config}"))
display(Markdown(f"Be sure to check the appropriate git hash -- {ts_config_ocs_githash}"))
display(HTML(f'<a href="{link}" target="_blank" rel="noreferrer noopener">{link}</a>')) # why does USDF RSP trap us?? can't click ..

Found 3 scheduler configuration records


,salIndex,classname,description,config,script_salIndex,timestampProcessStart,finalScriptState
2024-11-26 18:21:36.058000+00:00,1,Obsenv Update,ts_config_ocs: v0.25.2.alpha.2-245-geb0954e su...,ts_standardscripts: v1.38.0-113-g704598f ts_ex...,-1,2024-11-26 18:21:36.058000,Configuration
2024-11-27 01:06:04.858510+00:00,1,Scheduler dependencies,feature_scheduler 3.3.0,"cloudModel: 3.3.0, downtimeModel: 3.3.0, obser...",-1,2024-11-27 01:06:04.858510,Configuration
2024-11-27 01:06:04.859344+00:00,1,Scheduler configuration,ts_config_ocs v0.25.2.alpha.2-245-geb0954e v7,"_init.yaml,_summit.yaml,maintel_fbs_comcam_sv_...",-1,2024-11-27 01:06:04.859344,Configuration


fbs configuration yaml @ maintel_fbs_comcam_sv_survey.yaml

Be sure to check the appropriate git hash -- 0.25.2.alpha.2-245-geb0954e

In [64]:
# From the git hash, and the file 
# https://github.com/lsst-ts/ts_config_ocs/blob/eb0954ef5b43d661f2c03d7ec0c27301561c12d4/Scheduler/feature_scheduler/maintel/fbs_config_minimal_fieldsurvey.py
# we can see the expected sequence was:
# nvisits = {"u_02": 8, "g_01": 16, "r_03": 8, "i_06": 8, "z_03": 8, "y_04": 8}
# sequence = ["g_01", "r_03", "i_06"]
# # exposure time in seconds
# exptimes = {"u_02": 38, "g_01": 30, "r_03": 30, "i_06": 30, "z_03": 30, "y_04": 30}
# # 1 --> single 30 second exposure
# nexps = {"u_02": 1, "g_01": 1, "r_03": 1, "i_06": 1, "z_03": 1, "y_04": 1}

# We got 8 in r band - yes.
# We got 12 in g band -- nope  (expect 16)
# We got 14 in i band -- combination of two sequences (8 from first, would have 6 from second .. then there was more g queued up

In [43]:
# Get targets from the EFD 
# also get snapshots .. we can interleave maybe

salindex = 1

topic = 'lsst.sal.Scheduler.logevent_target'
fields = await efd_client.get_fields(topic)
targets = await efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
delta_target = (targets.index.values[1:] - targets.index.values[:-1]) / np.timedelta64(1, 's')
targets['delta_target'] = np.concatenate([np.array([0]), delta_target])
targets['count'] = np.arange(0, len(targets))
print(len(targets))

# Check how many snapshots too as this equals number of calls
topic = "lsst.sal.Scheduler.logevent_largeFileObjectAvailable"
fields = ["url"]
snapshots = await efd_client.select_time_series(topic, fields, t_start, t_end, index=salindex)
print(len(snapshots))
      
#print(fields)
tcols = ['delta_target', 'requestMjd', 'ra', 'decl', 'filter', 'exposureTimes0', 'slewTime', 'skyAngle', 'airmass', 'moonRa', 'skyBrightness', 'seeing', 'note', 'targetName', 'targetId', 'count']

pd.concat([snapshots, targets[tcols]]).sort_index()


40
1


,url,delta_target,requestMjd,ra,decl,filter,exposureTimes0,slewTime,skyAngle,airmass,moonRa,skyBrightness,seeing,note,targetName,targetId,count
2024-11-27 01:06:47.245930+00:00,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-11-27 01:06:48.736429+00:00,NaN,0.000000,0.0,38.352611,7.094078,r_03,30.0,3.043314,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,0.0
2024-11-27 01:06:49.922608+00:00,NaN,1.186179,0.0,37.975071,6.491146,r_03,30.0,3.899728,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,1.0
2024-11-27 01:07:51.358791+00:00,NaN,61.436183,0.0,37.367630,6.865412,r_03,30.0,3.899712,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,2.0
2024-11-27 01:08:41.493423+00:00,NaN,50.134632,0.0,37.744689,7.468826,r_03,30.0,3.759334,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,3.0
2024-11-27 01:09:33.798498+00:00,NaN,52.305075,0.0,38.393965,7.112703,r_03,30.0,3.211927,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,4.0
2024-11-27 01:10:25.918547+00:00,NaN,52.120049,0.0,38.016360,6.509805,r_03,30.0,3.875482,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,5.0
2024-11-27 01:11:20.287855+00:00,NaN,54.369308,0.0,37.408925,6.884123,r_03,30.0,3.875674,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,6.0
2024-11-27 01:12:14.400119+00:00,NaN,54.112264,0.0,37.786055,7.487505,r_03,30.0,3.745746,211.835772,1.406141,0.0,20.965915,0.0,,,0.0,7.0
2024-11-27 01:13:06.510376+00:00,NaN,52.110257,0.0,38.352611,7.094078,g_01,30.0,120.000000,211.835772,1.406141,0.0,21.906155,0.0,,,0.0,8.0


In [52]:
# Can pull up the snapshot, but given the configuration and the targets above, it seems likely that some requested observations got dropped by ts_scheduler
snapshot_index = 0

snapshot_time = snapshots.iloc[snapshot_index].name
snapshot_mjd = Time(snapshot_time).mjd
url = snapshots.iloc[snapshot_index].url
print(snapshot_time, url)

# EFD records the summit LFA -- if not at the summit, swap.
bucket = "s3://rubin:"
uri = ResourcePath(bucket + urlparse(url).path.lstrip('/'))

# multi-tenant names have colons 
os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"

try:
    result = uri.read()
except FileNotFoundError:
    print("OH NO")
    print(f"Snapshot {uri} seems to be missing")
    result = None

if result is not None:
    #unpickle
    sched, conditions = pickle.loads(result)

    # Just check that these are the right kind of things 
    assert isinstance(sched, CoreScheduler)
    assert isinstance(conditions, Conditions)

2024-11-27 01:06:47.245930+00:00 https://s3.cp.lsst.org/rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/26/Scheduler:1_Scheduler:1_2024-11-27T01:07:23.903.p
OH NO
Snapshot s3://rubin:rubinobs-lfa-cp/Scheduler:1/Scheduler:1/2024/11/26/Scheduler:1_Scheduler:1_2024-11-27T01:07:23.903.p seems to be missing


In [9]:
#sched.survey_lists

In [53]:
# Check conditions time, and visit to the target 
# print('conditions time', conditions.mjd) 
# # Check on alt/az limits in the conditions (to see if anything masked)
# print('tel limits', np.degrees(conditions.tel_alt_limits), np.degrees(conditions.tel_az_limits))
# # And check on current position of the telescope (especially due to this FBS configuration)
# print('tel pos', np.degrees(conditions.tel_ra), np.degrees(conditions.tel_dec))
# # Filters?
# print(conditions.current_filter, conditions.mounted_filters)

In [54]:
# just check on surveys and how many observations they know about .. 
# (there's only one tier in this scheduler configuration)
# Might as well check total # of visits against consdb previous visits 
# (since this is start of the day, these should be all from previous nights)

# visit_query = f'''SELECT * FROM cdb_{instrument}.visit1 where day_obs < {day_obs_int} and science_program like '%%BLOCK-320%%' '''
# visits = consdb.query(visit_query).to_pandas()

# for s in sched.survey_lists[0]:
#     # ok, the list is long .. only do this for the ones with non-zero obs for the survey .. 
#     if s.extra_features['ObsRecorded_note'].feature > -1:
#         print("survey", s.survey_name, np.degrees(s.ra), np.degrees(s.dec))
#         # hmm .. not all previous visits are counted .. but perhaps some of the previous visits were bad?
#         print("nvisits", s.extra_features['ObsRecorded'].feature, s.extra_features['ObsRecorded_note'].feature, len(visits.query('target_name == @s.survey_name')))

In [56]:
# Check times, looking at lag between visits and target and conditions
# visitidx = 100
# targetidx = 3
# print(conditions.mjd, vv.exp_midpt_mjd[visitidx], Time(targets.index[targetidx]).utc.mjd)
# print("first visit - conditions time")
# print((vv.exp_midpt_mjd[visitidx] - conditions.mjd) * days_to_seconds )
# print("conditions - target recorded time")
# print((conditions.mjd - Time(targets.index[targetidx]).utc.mjd) * days_to_seconds)

In [57]:
# All right - let's go update the scheduler to the conditions, and ask for an observation. 
# print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)
# sched.update_conditions(conditions)
# print(sched.conditions.mjd, conditions.mjd, sched.conditions.tel_ra)

# Also, given the times above .. the conditions.mjd is not quite the time the visit is acquired, 
# but it's also not the time the target is recorded .. so let's assume it's the goal visit start time.

In [58]:
# Find observations that the scheduler would request
# obs = sched.request_observation(conditions.mjd, whole_queue=True)

In [60]:
# print(len(obs))
# This is a hack to check which survey was the winner, since whole_queue clears the previous winner .. 
# but it also only works for the simple survey configuration in use here (because assume only tier, etc)
# for i, s in enumerate(sched.survey_lists[0]):
#     if s.target_name == obs[0]['target_name']:
#         print(s.target_name, i)
# for o in obs:
#     print(o['target_name'], o['filter'], np.degrees(o['rotTelPos']), np.degrees(o['rotSkyPos']))

In [65]:
# so -- today we have no snapshot because LFA replication is down. 
# However, I did happen to check this dither detailer in a simulation earlier and it looked fine.

# My suspicion is more that out of the intended 16 visits in g band, since there are 14 targets -- two got rejected in ts_scheduler.
# Let's see why.

In [92]:
# Assume the FBS asked for the correct visits ..
# Let's check the ts_scheduler logs where we can see if requested observations get turned into targets.

# the logs are plentiful, so let's zoom in to the time of the first good sequence - the time around target #3
tstart = t_start - TimeDelta(0, format='sec')
tend = t_end

tstart = Time('2024-11-27 01:13:05.491990') - TimeDelta(1, format='sec')
tend = Time('2024-11-27 01:37:54.576560')


topic = 'lsst.sal.Scheduler.logevent_logMessage'
fields = ['functionName', 'level', 'lineNumber' ,'message', 'salIndex']
log = await efd_client.select_time_series(topic, fields, tstart, tend, index=1)
display(HTML(log.to_html()))

# There are a lot of the scheduler rejecting the i-band targets from the first two sequences, with error 64 .. 
# Tracking this back, it's error  64 from ts_observatory_model, indicating that the filter change is not possible. 
# The reason for this isn't obvious .. filter burst seems most likely, but the filter_burst_time seems like it's set to 0, so should be fine.
# the average filter numbers are also averaged over a year, with a limit of 30,000 .. which also seems unlikely to have been hit.
# The name of the filter seems fine too (and executed in the last sequence). 

,functionName,level,lineNumber,message,salIndex
2024-11-27 01:13:05.486399+00:00,check_scheduled_targets,20,786,"Check scheduled report:\n\n\n\t observation completed successfully. Registering observation.\n\nFull report: \n\ttargetid=0 field=0 filter=r_03 exp_times=[30.0] ra=37.786 dec=7.488 ang=211.836 alt=45.165 az=36.836 rot=-0.342 telalt=45.165 telaz=36.836 telrot=-0.344 time=0.0 airmass=1.406 brightness=20.966 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=3.746 cost=0.000 rank=0.000 note= scripts still executing.\nname='Imaging' id=UUID('c6a7ea35-ead5-4432-8a78-ba95b3e97aec') program='BLOCK-320' constraints=[] scripts=[ObservingScript(name='maintel/track_target_and_take_image_comcam.py', standard=True, parameters={'ra': '+2:31:08.65315784', 'dec': '+7:29:15.01702982', 'rot_sky': 211.83577228951634, 'band_filter': 'r_03', 'name': 'Rubin_SV_38_7', 'estimated_slew_time': 3.74574611063018, 'exp_times': [30.0], 'num_exp': 1, 'obs_time': 0.0, 'program': 'BLOCK-320', 'reason': 'science', 'targetid': 0})] configuration_schema=''\n[107293]::[<ScriptState.CONFIGURED: 2>]..",1
2024-11-27 01:13:05.486558+00:00,remove_block_done_scripts,20,675,Removing [107292] done scripts from block c6a7ea35-ead5-4432-8a78-ba95b3e97aec.,1
2024-11-27 01:13:05.491990+00:00,put_on_queue,20,895,Adding target=targetid=0 field=0 filter=g_01 exp_times=[30.0] ra=38.353 dec=7.094 ang=211.836 alt=45.202 az=37.808 rot=0.419 telalt=45.202 telaz=37.808 telrot=0.417 time=0.0 airmass=1.406 brightness=21.906 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=120.000 cost=0.000 rank=0.000 note= scripts on the queue.,1
2024-11-27 01:13:05.509527+00:00,put_on_queue,20,899,Imaging::sal_index=107294.,1
2024-11-27 01:14:00.605047+00:00,check_scheduled_targets,20,786,"Check scheduled report:\n\n\n\t observation completed successfully. Registering observation.\n\nFull report: \n\ttargetid=0 field=0 filter=g_01 exp_times=[30.0] ra=38.353 dec=7.094 ang=211.836 alt=45.202 az=37.808 rot=0.419 telalt=45.202 telaz=37.808 telrot=0.417 time=0.0 airmass=1.406 brightness=21.906 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=120.000 cost=0.000 rank=0.000 note= scripts still executing.\nname='Imaging' id=UUID('c6a7ea35-ead5-4432-8a78-ba95b3e97aec') program='BLOCK-320' constraints=[] scripts=[ObservingScript(name='maintel/track_target_and_take_image_comcam.py', standard=True, parameters={'ra': '+2:33:24.6266478', 'dec': '+7:05:38.67980162', 'rot_sky': 211.83577228951637, 'band_filter': 'g_01', 'name': 'Rubin_SV_38_7', 'estimated_slew_time': 120.0, 'exp_times': [30.0], 'num_exp': 1, 'obs_time': 0.0, 'program': 'BLOCK-320', 'reason': 'science', 'targetid': 0})] configuration_schema=''\n[107294]::[<ScriptState.CONFIGURED: 2>]..",1
2024-11-27 01:14:00.605216+00:00,remove_block_done_scripts,20,675,Removing [107293] done scripts from block c6a7ea35-ead5-4432-8a78-ba95b3e97aec.,1
2024-11-27 01:14:00.610626+00:00,put_on_queue,20,895,Adding target=targetid=0 field=0 filter=g_01 exp_times=[30.0] ra=37.975 dec=6.491 ang=211.836 alt=45.912 az=37.814 rot=0.379 telalt=45.912 telaz=37.814 telrot=0.377 time=0.0 airmass=1.406 brightness=21.906 cloud=0.00 seeing=0.00 visits=0 progress=0.00% seqid=0 ssname= groupid=0 groupix=0 firstdd=False ddvisits=0 need=0.000 bonus=0.000 value=0.000 propboost=1.000 propid=[] need=[] bonus=[] value=[] propboost=[] slewtime=120.000 cost=0.000 rank=0.000 note= scripts on the queue.,1
2024-11-27 01:14:00.813851+00:00,put_on_queue,20,899,Imaging::sal_index=107295.,1
2024-11-

In [97]:
# i think this means that there were 16 initial observation -> target transformations (matching observations expected)
print(len(log.query('functionName == "put_on_queue" and message.str.startswith("Adding target") and message.str.contains("g_01")')))

16


In [100]:
# but tiago tells me some of the messages above indicate faults -- and yes, there were faults during this time period 
async def get_error_codes(t_start: Time, t_end: Time, efd_client: EfdClient) -> pd.DataFrame:
    """Get all messages from logevent_errorCode topics."""
    # Get error codes
    topics = await efd_client.get_topics()
    err_codes = [t for t in topics if 'errorCode' in t]
    
    errs = []
    for topic in err_codes:
        df = await efd_client.select_time_series(topic, ['errorCode', 'errorReport'], t_start, t_end)
        if len(df) > 0:
            df['topic'] = topic
            errs += [df]
    if len(errs) > 0:
        errs = pd.concat(errs).sort_index()
        def strip_csc(x):
            return x.topic.replace("lsst.sal", "").replace("logevent_errorCode", "").replace(".", "") + "CSC error"
        errs['component'] = errs.apply(strip_csc, axis=1)
        # Rename some columns to match narrative log columns
        errs.rename({'errorCode': 'error_code', 'errorReport': 'message_text', 'topic': 'origin'}, axis=1, inplace=True)
        # Add a salindex so we can color-code based on this as a "source"
        errs['salIndex'] = 4
        errs['finalStatus'] = "ERR"
        errs['timestampProcessStart'] = errs.index.values.copy()
    
    print(f"Found {len(errs)} error messages")
    return errs

errs = await get_error_codes(t_start, t_end, efd_client)
# g band only 
errs = await get_error_codes(tstart, tend, efd_client)
errs

Found 12 error messages
Found 8 error messages


,error_code,message_text,origin,component,salIndex,finalStatus,timestampProcessStart
2024-11-27 01:18:19.694721+00:00,1,Low-level controller went to FAULT state,lsst.sal.MTRotator.logevent_errorCode,MTRotatorCSC error,4,ERR,2024-11-27 01:18:19.694721
2024-11-27 01:18:19.721931+00:00,6651,Error Rotator replied error for trackTarget co...,lsst.sal.MTPtg.logevent_errorCode,MTPtgCSC error,4,ERR,2024-11-27 01:18:19.721931
2024-11-27 01:19:40.830468+00:00,0,,lsst.sal.MTRotator.logevent_errorCode,MTRotatorCSC error,4,ERR,2024-11-27 01:19:40.830468
2024-11-27 01:19:46.393328+00:00,0,,lsst.sal.MTPtg.logevent_errorCode,MTPtgCSC error,4,ERR,2024-11-27 01:19:46.393328
2024-11-27 01:31:05.767398+00:00,1,Low-level controller went to FAULT state,lsst.sal.MTRotator.logevent_errorCode,MTRotatorCSC error,4,ERR,2024-11-27 01:31:05.767398
2024-11-27 01:31:05.792282+00:00,6651,Error Rotator summaryState Fault,lsst.sal.MTPtg.logevent_errorCode,MTPtgCSC error,4,ERR,2024-11-27 01:31:05.792282
2024-11-27 01:31:41.756105+00:00,0,,lsst.sal.MTPtg.logevent_errorCode,MTPtgCSC error,4,ERR,2024-11-27 01:31:41.756105
2024-11-27 01:32:08.674753+00:00,0,,lsst.sal.MTRotator.logevent_errorCode,MTRotatorCSC error,4,ERR,2024-11-27 01:32:08.674753


In [101]:
# and there were two failed scripts as a result -- so two visits were lost to rotator failures
# and if these failures also took with them a second target (because the queue works in twos at this level)
# then 16 - 2 (failed scripts) - 2 (vanished targets due to the errors0 = 12 visits. 

So I think we've accounts for the fact that there were four missing requested visits, and it was just unlucky as to where they were.